# Client Financial Analysis Notebook

This notebook integrates all key financial formulas and reusable code to analyze any client by their ID. It is structured for business and risk analysis in B2B/ERP lending portfolios.

## 1. Import Required Libraries
Import pandas, numpy, and any other libraries needed for data manipulation and analysis.

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

# Optional: display settings for pandas
display_cols = 20
pd.set_option('display.max_columns', display_cols)
pd.set_option('display.float_format', lambda x: f'{x:,.2f}')

## 2. Define Financial Analysis Formulas
Implement Python functions for the financial formulas used in client analysis (APR, yield, DPD, rotation, ticket, recurrence, etc.).

In [ ]:
# --- Financial Formulas for Client Analysis ---
def weighted_average(series, weights):
    return np.average(series, weights=weights) if len(series) > 0 else np.nan

def apr_annual_nom(apr_m):
    return apr_m * 12

def apr_annual_eff(apr_m):
    return (1 + apr_m) ** 12 - 1

def yield_per_term(apr_m, term):
    return (1 + apr_m) ** (term / 30) - 1

def rotation(avg_term):
    return 360 / avg_term if avg_term > 0 else np.nan

def line_up(disb_first, disb_last):
    return disb_last / disb_first - 1 if disb_first > 0 else np.nan

def rate_down(apr_first, apr_last):
    return apr_last - apr_first

def tenor_up(term_first, term_last):
    return term_last - term_first

def dpd_proxy(days_realized, term):
    return days_realized - term

def percent_back_to_back(days_between, term):
    return np.mean(days_between <= (term + 15)) if len(days_between) > 0 else np.nan

## 3. Load Client Data
Load the client dataset from a CSV file or database into a pandas DataFrame.

In [ ]:
# Example: Load loan data from CSV
# Update the path as needed for your environment
data_path = '../data/loans.csv'  # Change to your actual data location
loans = pd.read_csv(data_path, parse_dates=['disbursement_date'])

# Preview data
loans.head()

## 4. Input Client ID for Analysis
Prompt the user to input a client ID or select one from the dataset for analysis.

In [ ]:
# List unique client IDs for selection
client_ids = loans['customer_id'].unique()
print(f"Available client IDs: {client_ids}")

# Input client ID for analysis
client_id = input("Enter the client ID to analyze: ")

## 5. Apply Formulas to Selected Client
Filter the DataFrame for the selected client ID and apply the defined formulas to compute relevant metrics.

In [ ]:
# Filter loans for selected client
client_loans = loans[loans['customer_id'] == client_id].copy()

# Compute metrics using the defined formulas
results = {}

# Volumen y exposición
total_disbursed = client_loans['disbursement_amount'].sum()
current_outstanding = client_loans['outstanding'].sum()
share_portfolio = current_outstanding / loans['outstanding'].sum()
results['Total Disbursed'] = total_disbursed
results['Current Outstanding'] = current_outstanding
results['Share in Portfolio'] = share_portfolio

# Rotación y tenor
avg_term = weighted_average(client_loans['term'], client_loans['disbursement_amount'])
results['Avg Term (days)'] = avg_term
results['Rotation'] = rotation(avg_term)

# APR contractual
apr_m = weighted_average(client_loans['APR_m'], client_loans['disbursement_amount'])
results['APR_m (monthly)'] = apr_m
results['APR_annual_nom'] = apr_annual_nom(apr_m)
results['APR_annual_eff'] = apr_annual_eff(apr_m)

# Máx y promedio DPD
results['Max DPD'] = client_loans['DPD'].max()
results['Mean DPD'] = client_loans['DPD'].mean()
results['% Loans DPD>30'] = (client_loans['DPD'] > 30).mean()
results['% Loans DPD>60'] = (client_loans['DPD'] > 60).mean()
results['% Loans DPD>90'] = (client_loans['DPD'] > 90).mean()

# Estado actual (PAR)
results['PAR30'] = (client_loans['DPD'] > 30).sum() / current_outstanding if current_outstanding > 0 else np.nan
results['PAR60'] = (client_loans['DPD'] > 60).sum() / current_outstanding if current_outstanding > 0 else np.nan
results['PAR90'] = (client_loans['DPD'] > 90).sum() / current_outstanding if current_outstanding > 0 else np.nan

# Frecuencia y ticket
client_loans = client_loans.sort_values('disbursement_date')
client_loans['days_between'] = client_loans['disbursement_date'].diff().dt.days
results['Median Days Between'] = client_loans['days_between'].median()
results['P75 Days Between'] = client_loans['days_between'].quantile(0.75)
results['% Back-to-Back'] = percent_back_to_back(client_loans['days_between'].dropna(), client_loans['term'].mean())
results['Avg Ticket'] = client_loans['disbursement_amount'].mean()

# Evolución de condiciones
if len(client_loans) > 1:
    first = client_loans.iloc[0]
    last = client_loans.iloc[-1]
    results['Line Up'] = line_up(first['disbursement_amount'], last['disbursement_amount'])
    results['Rate Down'] = rate_down(first['APR_m'], last['APR_m'])
    results['Tenor Up'] = tenor_up(first['term'], last['term'])
else:
    results['Line Up'] = results['Rate Down'] = results['Tenor Up'] = np.nan

# Recurrencia y concentración
results['Repeat 60d'] = (client_loans['days_between'] <= 60).any()
results['Concentration 12M'] = client_loans[client_loans['disbursement_date'] > '2024-01-01']['disbursement_amount'].sum() / loans[loans['disbursement_date'] > '2024-01-01']['disbursement_amount'].sum()

# Display results as DataFrame
results_df = pd.DataFrame(results, index=[client_id])

## 6. Display Analysis Results
Present the computed metrics and analysis results for the selected client in a clear, tabular format.

In [ ]:
# Display the results
display(results_df)

# Optionally, print a summary message for the board
summary = (
    f"{client_id} representa ~{results['Share in Portfolio']:.1%} del AUM, "
    f"con un APR contractual anualizado de ~{results['APR_annual_nom']:.1%} "
    f"y un yield efectivo de ~[calcular_yield_efectivo]. "
    f"Es un cliente de [alto / medio / bajo] margen relativo al promedio de la cartera."
)
print(summary)

In [ ]:
# --- Completa las métricas ---
# Yield efectivo (cash, por loan)
if 'Interest' in client_loans.columns and 'Fees' in client_loans.columns and 'Other' in client_loans.columns and 'Rebates' in client_loans.columns:
    client_loans['Yield_term'] = (client_loans['Interest'] + client_loans['Fees'] + client_loans['Other'] - client_loans['Rebates']) / client_loans['disbursement_amount']
    results['Yield efectivo (media)'] = client_loans['Yield_term'].mean()
    results['Yield efectivo (ponderado)'] = weighted_average(client_loans['Yield_term'], client_loans['disbursement_amount'])
else:
    results['Yield efectivo (media)'] = results['Yield efectivo (ponderado)'] = np.nan

# DPD Proxy percentiles
if 'days_realized' in client_loans.columns and 'term' in client_loans.columns:
    client_loans['DPD_proxy'] = client_loans['days_realized'] - client_loans['term']
    results['DPD_proxy_p50'] = client_loans['DPD_proxy'].median()
    results['DPD_proxy_p90'] = client_loans['DPD_proxy'].quantile(0.9)
else:
    results['DPD_proxy_p50'] = results['DPD_proxy_p90'] = np.nan

# Actualiza el DataFrame de resultados
results_df = pd.DataFrame(results, index=[client_id])

display(results_df)

# --- Visualización de análisis de cliente ---
import matplotlib.pyplot as plt
import seaborn as sns

# Distribución de DPD y Yield
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Histograma de DPD
sns.histplot(client_loans['DPD'], bins=20, kde=True, ax=axes[0], color='skyblue')
axes[0].set_title('Distribución de DPD')
axes[0].set_xlabel('DPD (días)')
axes[0].set_ylabel('Número de préstamos')

# Histograma de Yield efectivo
if 'Yield_term' in client_loans.columns:
    sns.histplot(client_loans['Yield_term'], bins=20, kde=True, ax=axes[1], color='orange')
    axes[1].set_title('Distribución de Yield efectivo')
    axes[1].set_xlabel('Yield efectivo por préstamo')
    axes[1].set_ylabel('Número de préstamos')
else:
    axes[1].set_visible(False)

plt.tight_layout()
plt.show()

# Evolución temporal de APR y DPD
fig, ax1 = plt.subplots(figsize=(10, 5))

if 'APR_m' in client_loans.columns:
    ax1.plot(client_loans['disbursement_date'], client_loans['APR_m'], marker='o', label='APR mensual', color='blue')
    ax1.set_ylabel('APR mensual', color='blue')
    ax1.tick_params(axis='y', labelcolor='blue')

if 'DPD' in client_loans.columns:
    ax2 = ax1.twinx()
    ax2.plot(client_loans['disbursement_date'], client_loans['DPD'], marker='x', label='DPD', color='red')
    ax2.set_ylabel('DPD', color='red')
    ax2.tick_params(axis='y', labelcolor='red')

plt.title(f'Evolución temporal de APR y DPD para {client_id}')
plt.show()

In [ ]:
# --- Alertas Potenciales por Cliente (Riesgo Anchor Amber/Red) ---

def anchor_concentration_alert(invoices, client_id, threshold=0.4):
    client_inv = invoices[invoices['customer_id'] == client_id]
    anchor_share = client_inv.groupby('anchor_id')['invoice_amount'].sum() / client_inv['invoice_amount'].sum()
    max_anchor = anchor_share.idxmax() if not anchor_share.empty else None
    max_share = anchor_share.max() if not anchor_share.empty else 0
    alert = max_share > threshold
    return alert, max_anchor, max_share

def credit_line_usage_alert(credit_used, credit_limit, threshold=0.9):
    usage = credit_used / credit_limit if credit_limit > 0 else 0
    alert = usage > threshold
    return alert, usage

def extended_term_alert(loans, client_id, threshold=90):
    client_loans = loans[loans['customer_id'] == client_id]
    long_terms = client_loans[client_loans['term'] > threshold]
    alert = not long_terms.empty
    return alert, long_terms

def apr_gap_alert(loans, client_id, bps_gap=0.05):
    client_loans = loans[loans['customer_id'] == client_id]
    if 'APR_effective' in client_loans and 'APR_contractual' in client_loans:
        gap = client_loans['APR_contractual'] - client_loans['APR_effective']
        alert = (gap > bps_gap).any()
        return alert, gap
    return False, None

# --- Ejemplo de uso para un cliente ---
# invoices: DataFrame con ['customer_id', 'anchor_id', 'invoice_amount']
# loans: DataFrame con ['customer_id', 'term', 'APR_effective', 'APR_contractual']
# credit_used, credit_limit: valores numéricos para el cliente

# 1. Concentración con anchor
alert1, anchor, share = anchor_concentration_alert(invoices, client_id)
if alert1:
    print(f"Alerta: >40% de facturas con anchor {anchor} ({share:.1%}). Recomendar diversificar y limitar exposición.")

# 2. Uso de línea de crédito
alert2, usage = credit_line_usage_alert(credit_used, credit_limit)
if alert2:
    print(f"Alerta: Uso de línea >90% ({usage:.1%}). Revisar capacidad y monitorear liquidez.")

# 3. Plazos extendidos
alert3, long_terms = extended_term_alert(loans, client_id)
if alert3:
    print(f"Alerta: Plazos de pago >90 días detectados. Negociar plazos más cortos.")
    display(long_terms[['term']])

# 4. Caída en rendimiento
alert4, gap = apr_gap_alert(loans, client_id)
if alert4:
    print(f"Alerta: APR efectivo >500 bps menor al contractual. Revisar pricing y ajustar tasas.")
    display(gap)


# Lineamientos y Acciones de Política Crediticia para Optimizar Riesgo-Retorno

**Advance Rate según Riesgo:**
- AAA/Anchors top: Advance rate 90–95% del valor factura.
- Medio riesgo: Advance 80–85%.
- Alto riesgo/no verificados: Advance 70–75%.
- _Acción:_ Ajustar advance automáticamente según rating del anchor/pagador y cliente. Retener más en pagadores/segmentos con historial de mora o sin track record.

**Límites por Anchor (Pagador):**
- Límite máximo: 20% del portafolio por anchor; 10% para anchors medianos o menor calificación.
- _Acción:_ Monitorear concentración y rechazar nuevas operaciones que excedan el umbral, o buscar cofinanciadores/seguros.

**Límites por Cliente (Proveedor):**
- Límite máximo: 10% del patrimonio de Ábaco o monto absoluto según capacidad de absorción de pérdidas.
- Graduación: Líneas más bajas para clientes B–C y mínimas para D–E.
- _Acción:_ Automatizar alertas y revisiones de línea conforme crecen los saldos.

**Pricing Mínimo por Bucket de Riesgo:**
- A: APR mínima 25%, comisión 1–2%.
- B–C: APR mínima 30%, comisión estándar (~2%).
- D–E: APR mínima 35–40%, comisión 3%+.
- F–H: APR ≥50% o caso a caso.
- _Acción:_ Estandarizar mínimos en el sistema para evitar guerra de precios y proteger margen.

**Políticas de Renovación/Evergreen:**
- Pausa obligatoria tras 6 meses de uso continuo, forzar liquidación y recalificación.
- Condición: Mantener DSO bajo control para continuidad evergreen.

---

## Acciones por Cliente Activo
- **Clientes Top:** Upsell controlado, aumentar línea si usan <70%, ofrecer mejores condiciones, monitoreo estándar.
- **Clientes Watchlist:** No upsell, reforzar cobro, seguimiento cercano, exigir info financiera mensual.
- **Clientes Run-off:** No nuevas operaciones, foco en recuperación, documentar aprendizajes.
- **Nuevos Potenciales:** Prospectar clones de los mejores, pre-aprobar lineamientos para perfiles ideales.
- **Pagadores con Banderas Rojas:** Limitar/descontinuar financiamiento, monitoreo de noticias, listas restringidas.

---

## Perfil del Cliente Ideal
Empresa mediana, historial financiero y crediticio sólido, proveedora de varios corporativos grandes, uso regular y disciplinado de la línea, buena documentación y transparencia.

---

## Conclusión
Implementar estas políticas y acciones permite:
- Maximizar ingresos con clientes top (más negocio seguro, mejores tasas).
- Minimizar pérdidas con clientes riesgosos (límites, pricing, monitoreo).
- Fortalecer la cultura de riesgo y la estabilidad del portafolio.

# Net Interest Margin After Loss (NIMAL)

**Definición:** Margen financiero neto de Ábaco una vez descontado el costo directo de fondeo y las pérdidas crediticias (realizadas o provisionadas).

**Fórmulas:**
- **NIMAL (sobre costo):**
  $$(\text{Ingreso financiero neto} - \text{pérdidas}) / \text{Costo directo}$$
- **NIMAL (sobre AUM):**
  $$(\text{Ingreso financiero neto} - \text{pérdidas}) / \text{AUM promedio}$$

**Interpretación ejecutiva:**
- **2024:** NIMAL after loss (sobre costo directo): ~80.2%  
  Por cada USD 1 de costo directo (funding + crédito), Ábaco genera ~0.80 USD de margen neto después de pérdidas.
- **2025 YTD:** NIMAL after loss (sobre costo directo): ~66.6%  
  El margen sigue alto, pero cae vs 2024 por mayor costo marginal de fondeo, scaling de estructura y mayor peso de plazos largos/riesgo.

**Conexión con cartera y KPIs:**
- **NIMAL_AUM:**
  $$(\text{Interest income} + \text{fees} - \text{funding cost} - \text{credit losses}) / \text{Average AUM}$$
- **Relación con APR efectivo:**
  NIMAL after loss = APR efectivo – costo de fondeo – tasa de pérdida realizada.

**Para inversionistas:**
- NIMAL es un KPI clave para mostrar la rentabilidad real de la cartera después de fondeo y riesgo.
- 2024: ~80.2% sobre costo directo.
- 2025 YTD: ~66.6% sobre costo directo.

_Estos ratios están listos para mostrarse como “NIMAL (post-loss)” en el deck y pueden alinearse con el APR efectivo y otros KPIs de tu loan tape._

In [ ]:
# Cálculo de NIMAL_AUM usando EEFF y cartera
import pandas as pd

# Ruta al archivo EEFF consolidado (ajusta según tu estructura)
eeff_path = '../data/EEFF ABACO CONSOLIDADO - ESTADO DE RESULTADOS - SEP-25.xlsx'
pyg = pd.read_excel(eeff_path, sheet_name='PYG', header=None)

# Extrae ingresos, costo directo y pérdidas (ajusta filas/columnas según tu archivo)
ingresos = pyg.iloc[11, 2]  # Fila 12, columna TOTAL 2024 (ajusta si es necesario)
costo_directo = pyg.iloc[12, 2]  # Fila 13, columna TOTAL 2024
perdidas = pyg.iloc[13, 2]  # Fila 14, columna TOTAL 2024

# AUM promedio (puedes calcularlo desde tu loan tape o EEFF)
aum_inicio = 5000000  # Ejemplo: reemplaza con tu dato real
aum_fin = 7000000     # Ejemplo: reemplaza con tu dato real
aum_prom = (aum_inicio + aum_fin) / 2

# NIMAL sobre costo y sobre AUM
nimal_costo = (ingresos - perdidas) / costo_directo if costo_directo else None
nimal_aum = (ingresos - perdidas) / aum_prom if aum_prom else None

print(f"NIMAL (sobre costo directo): {nimal_costo:.2%}")
print(f"NIMAL (sobre AUM promedio): {nimal_aum:.2%}")